
# Summary

Today, we're going to be implementing least squares regression subject to a bound on *total variation*. That is, we're going to find a curve $\hat \mu$ on $[0,1]$ that solves this optimization problem.

$$
\begin{aligned}
\hat \mu &= \operatorname*{argmin}_{\substack{m \\ \rho_{TV}(m) \le B}} \frac{1}{n}\sum_{i=1}^n \{ Y_i - m(X_i) \}^2 \qquad \text{ where } \\  & \rho_{TV}(m) = \max_{\substack{\text{increasing sequences} \\ 0=x_1 \le x_2 \le \ldots \le x_k=1}}
                  \sum_{j=1}^{k-1} \left\lvert m(x_{j+1})-m(x_j) \right\rvert.
\end{aligned}
$$

This is going to be a lot like implementing monotone regression. In fact, I changed only three lines of my implementation of monotone regression. And two of those are boilerplate. I'll give you those, so ultimately the change you'll be writing is a one-liner.
But there is another issue that we didn't have to deal with before. We have to choose a sensible value of the variation budget $B$. We'll take a look at how this impacts the curve we get and consider *cross-validation* as a strategy for choosing it automatically.

We'll use a few libraries.

In [ ]:
suppressPackageStartupMessages({
    library(tidyverse)
    library(CVXR)
})

# OSQP claims some feasible problems aren't, so we'll tell CVXR not to use it
CVXR::add_to_solver_blacklist('OSQP')  

# And we'll style our plots  
theme_update(plot.background = element_rect(fill = "transparent", colour = NA),
		    panel.background = element_rect(fill = "transparent", colour = NA),
                    legend.background = element_rect(fill="transparent", colour = NA),
                    legend.box.background = element_rect(fill="transparent", colour = NA),
                    legend.key = element_rect(fill="transparent", colour = NA),
			panel.grid.major=element_line(color=rgb(1,0,0,.1,  maxColorValue=1)),
	        panel.grid.minor=element_line(color=rgb(0,0,1,.1,  maxColorValue=1)),
		    axis.ticks.x = element_blank(),
		    axis.ticks.y = element_blank(),
		    axis.text.x  = element_text(colour = "#aaaaaa"),
		    axis.text.y  = element_text(colour = "#aaaaaa"),
		    axis.title.x  = element_text(colour = "#aaaaaa"),
		    axis.title.y  = element_text(colour = "#aaaaaa", angle=90))

And the function `invert.unique` from the monotone regression lab. We'll be using this all the time. It takes a vector $X$ and returns a list of two things.

1. A vector `elements` cointaining the positions of the unique elements of $X$, so that `X[elements]` is vector of unique elements of $X$ sorted in increasing order. 
    - This vector has length $p$ where $p$ is the number of unique elements in $X$.
2. A vector `inverse` that tells you, for each $i$, the position of $X[i]$ in $X[elements]$. 
    - This vector has length $n$ where $n$ is the length of $X$.
    - `X[elements][inverse]` is the same as `X`.

In [ ]:
invert.unique = function(x) { 
  o = order(x)
  dup = duplicated(x[o])
  inverse = rep(NA, length(x))
  inverse[o] = cumsum(!dup)
  list(elements=o[!dup], inverse=inverse)
}

a = c(1,2,3,3,4,5,5,5,6)
unique.a=invert.unique(a)
stopifnot(a[unique.a$elements][unique.a$inverse] == a)

# Implementation

Much like we did with monotone regression, it'll help to start by solving for a function $\hat\mu_{\mathcal{X}}$ 
that 

RESTRICTION AND EXPANSION


with a constraint defined on the sample. In particular, we'll work with a seminorm $\hat\rho_{TV}$ which, instead of being defined as a maximum over all increasing sequences, uses a single increasing sequence: the observations $X_1 \le X_2 \le \ldots \le X_n$, which we will assume we have sorted into increasing order throughout. That is, we'll solve this finite-dimensional optimization problem.

$$
\begin{aligned}
\hat \mu &= \operatorname*{argmin}_{\substack{m \\ \hat\rho_{TV}(m) \le B}} 
            \frac{1}{n}\sum_{i=1}^n \{ Y_i - m(X_i) \}^2 \qquad \text{ where } \\ 
&\hat\rho_{TV}(m) = \sum_{i=1}^{n-1} \left\lvert m(X_{i+1})-m(X_i) \right\rvert.
\end{aligned}
$$ 

As in the analogous problem we discussed when doing monotone regression, both our squared error and our constraint depend only on the values $m(X_1) \ldots m(X_n)$ of $m$. And if we have the values $\hat\mu^{\text{sample}}(X_1) \ldots \hat\mu^{\text{sample}}(X_n)$ of a solution to this problem, we can easily find a solution to our original problem. In fact, you already have code to do it. Let me explain.

## From Bounded Variation on the Sample to Bounded Variation

The short answer is that we can take $\hat \mu(x)$ to be any curve that agrees with $\hat \mu^{\text{sample}}$ at $X_1 \ldots X_n$, is *monotone* between the observations, and is *constant* outside the range of the data. Typically, we use piecewise-constant interpolation, defining $\hat \mu(x)$ to be $\hat \mu^{\text{sample}}(X_i)$ where $X_i$ is the closest observation to the left of $x$ if $x$ is inside the range of the data and the leftmost observation otherwise: $X_i=\max\ \{X_i : X_i \le x\} \cup \{X_1\}$. That's what the function `predict.piecewise.constant` we've been using with monotone regression does.

Here's why this gives us a solution to our original problem. The explanation is based on three observations.

1.  Any curve we're considering in our original optimization also gets considered in our finite-dimensional analog. That is, $\hat\rho_{TV}(m) \le \rho_{TV}(m)$ for any curve $m$ and therefore $$ \left\{ m : \rho_{TV}(m) \le B\right\} \subseteq \left\{ m : \hat\rho_{TV}(m) \le B \right\} $$ We know this because $\hat\rho_{TV}(m)$ is the sum $\sum_{i=1}^{n-1} \left\lvert m(X_{i+1})-m(X_i) \right\rvert$ for a particular increasing sequence $X_1 \ldots X_n$ and $\rho_{TV}(m)$ the maximum of *the same sum* over all increasing sequences $X_1 \ldots X_n$.

2.  This implies that any curve $\hat\mu$ that solves our original optimization would be considered in our finite-dimensional analog, so the prediction error that we're minimizing in both problems, $\text{MSPE}(m)=\frac{1}{n}\sum_{i=1}^n \{ Y_i - m(X_i) \}^2$, is *at least as small* at a solution $\hat\mu_{\text{sample}}$ to the finite-dimensional version as it is in the original problem. Consequently, if a solution $\hat\mu_{\text{sample}}$ to our finite-dimensional problem satisfies the stronger constraint $\rho_{TV}(m) \le B$ needed to be considered in the original problem, it's also a solution to the original problem: it has the smallest $\text{MSPE}$ among curves in the set $\left\{ m : \hat\rho_{TV}(m) \le B \right\}$, so it must have the smallest $\text{MSPE}$ among any subset of those curves, including $\left\{ m : \rho_{TV}(m) \le B \right\}$.

3.  For any curve $m^{\text{sample}}$, we can find a curve $m$ that *agrees on the sample*, i.e. with $m(X_i) = m^{\text{sample}}(X_i)$, that has total variation *equal to* its sample total variation. The piecewise-constant curve we've been drawing is one of them: all of its variation is at the sample points. Furthermore, because it agrees with $m^{\text{sample}}$ on the sample, it has the same $\text{MSPE}$. That means the piecewise-constant curve we've proposed through the points $\hat\mu^{\text{sample}}(X_i)$ is a solution to the finite-dimensional problem and satisfies the stronger constraint imposed in the original problem. Thus, it's a solution to the original problem.

To reinforce your understanding of this argument, do this exercise. We'll work with a sample of three points $(X_i,Y_i)$. $$ \left\{\left(0, \frac{1}{2}\right), \left(\frac{1}{2},0\right), \left(1,\frac{1}{4}\right)\right\} $$You can think of what we'll be doing as stepping through our approach to bounded variation regression in a world in which there are only six curves: $m_1(x)=x^2$, $m_2(x) = 3(x-1/2)^2$, $m_3(x) = 2(x-3/4)^2$, and corresponding piecewise-constant curves $\tilde m_1$, $\tilde m_2$, and $\tilde m_3$


Draw each of these six curves. I suggest you draw three plots in total: one of $m_1$ and $\tilde m_1$ together, another of $m_2$ and $\tilde m_2$, and another of $m_3$ and $\tilde m_3$. For each, calculate the sample total variation $\hat\rho_{TV}(m)$ and the total variation $\rho_{TV}(m)$. 
Of these curves, which minimize mean squared prediction error among the curves with $\hat\rho_{TV}(m) \le 1$? And which minimize it among the curves with $\rho_{TV}(m) \le 1$?


## Solving the Finite-dimensional Problem

All this leaves is finding the values $\hat\mu_{\text{sample}}(X_i)$ of a solution to the finite-dimensional optimization problem. As in the Monotone Regression Lab, we can almost get away with substituting the elements $\vec m_i$ of a vector $\vec m \in \mathbb{R}^n$ for the function values $m(X_1) \ldots m(X_n)$. The one wrinkle is that we need to make sure these elements correspond to the values of a function, i.e., that the vector satisfies $\vec m_i = \vec m_j$ if $X_i=X_j$. That is,


$$
\hat \mu(X_i) = \vec \mu_i \ \text{ where } \ \vec \mu = 
\operatorname*{argmin}_{\substack{\vec m \in \mathbb{R}^n \\
                   \sum_{i=1}^{n-1} \left\lvert \vec m_{i+1} - \vec m_i\right\rvert \le B 
                   \\ \vec m_{i+1} = \vec m_i \ \text{ for all $i$ with $X_{i+1}=X_i$}}}               
                   \frac{1}{n}\sum_{i=1}^n (Y_i - \vec m_i)^2. 
$$


We covered this in the Optimized Implementation Section of the Monotone Regression Lab.

This is a very similar optimization to the one we used for monotone regression. We're still optimizing over a vector $\vec{m}$ of $n$ values with the interpretation $\vec{m}_i=m(X_i)$. We're still minimizing mean squared prediction error. All we've got to do is change our constraint. 
To save you some trouble, I've copied my monotone regression code into the block below. And I've handled some boilerplate changes for you, too.

1.  I've changed the name and arguments of the function (line 1).
2.  I've changed the 'class' attribute in the output so *R* knows it's a bounded variation regression rather than a monotone one (line 36).
3.  I've defined 'predict.bvreg' to do piecewise-constant interpolation like we did in the monotone regression lab (line 72).

That means that all you have to do is change the constraint. The one that's on line 22 in the 

In [ ]:
bvreg = function(X, Y, B=1) {
  # Step 0.
  # We check that the inputs satisfy our assumptions.
  stopifnot(length(X) == length(Y))
  input = list(X=X, Y=Y)
  n = length(X)
  # and find the unique elements of X and the inverse mapping
  unique.X = invert.unique(X)

  # Step 1.
  # We tell CVXR we're thinking about a vector of unknowns m in R^p.
  m = Variable(length(unique.X$elements))
  # and permute and duplicate these into a vector mX with n elements in correspondence with (X_1,Y_1)...(X_n,Y_n)
  mX = m[unique.X$inverse]

  # Step 2.
  # We tell CVXR that we're interested in mean squared error.
  mse = sum((Y - mX)^2 / n)

  # Step 3.
  # We specify our constraints.
  constraints = list( diff(m) >= 0 )

  # Step 4.
  # We ask CVXR to minimize mean squared error subject to our constraints.
  # And we ask for vector mu.hat that does it.
  solved = solve(Problem(Minimize(mse), constraints))
  mu.hat = solved$getValue(m)

  # Step 5: a little boilerplate to make it idiomatic R.
  # 1. we record the unique levels of X and mu.hat, in correspondence and sorted in increasing order of X, in a list. We also record the input data. 
  # 2. we assign that list a class, so R knows predict should delegate to predict.monotonereg
  # 3. we return the list
  model = list(X = X[unique.X$elements], mu.hat = mu.hat, input = input)
  attr(model, "class") = "bvreg"
  model
}

In [ ]:
# make predictions based on piecewise-constant interpolation
# we use the curve that jumps at each observation and is otherwise constant
# that is, if X[1] < X[2] < ..., 
#   mu.hat(x) for x between X[k] and X[k+1] is mu.hat(X[k])   [case 1]
#             for x > X[k]  is mu.hat(X[k])                   [case 2]
#             for x < X[1]  is mu.hat(X[1])                   [case 3]
predict.bvreg = function(model, newdata=data.frame(X=model$input$X)) {
  y = model$mu.hat; X=model$X; x=newdata$X
  # for each new data point x in newdata$X, 
  # find the closest observed X[k] left of x
  # i.e., the largest k for which X[k] <= x 
  # this covers cases 1 and 2
  # bin will be a vector of these numbers k, with one for each x in newdata$X
  bin = findInterval(x, X) 
  # if there is no X[k] < x, findInterval tells us k=0
  # to cover case 3, we want X[k] for k=1 when this happens.
  bin[bin==0] = 1
  # report the values of mu.hat(X[k]), one for each x
  y[bin]
}


TODO: Write some stuff here.

In [ ]:

prediction.function = function(model) {
  function(x) { predict(model, data.frame(X=x)) }
}


Once you've got code, try fitting some data to see that it works. You can just run this block, but it might be instructive to try changing the variation budget $B$ and look at how the fit changes. Or try changing the function $\mu$ from the step to something else and see how the fit changes.

In [ ]:
mu = function(x) { 1*(x >= .5) }
sigma = .5
n = 400
X = runif(n)
Y = mu(X) + sigma*rnorm(n)

x = seq(0,1,by=.001) 
mu.hat = prediction.function(bvreg(X,Y, B=1))
ggplot() + geom_point(aes(x=X,y=Y), alpha=.2) +
           geom_line(aes(x=x, y=mu(x)), alpha=.4) +
           geom_line(aes(x=x,y=mu.hat(x)), color='blue')


# Behavior

Now that we've got some bounded variation regression code, let's see how it works and how that depends on the variation budget $B$. We'll try fitting data sampled around four simple curves. Data that looks like this.

In [ ]:
n=200
sigma = .5
mus = list(step     = function(x) { 1*(x >= .5) },
	         line     = function(x) { x },
	         stepline = function(x) { x*(x >= .5) }, 
	         sin      = function(x) { sin(pi*x) })
           
x = seq(0,1,by=.01)
for(mu.name in names(mus)) { 
    mu = mus[[mu.name]]
    X     = runif(n)
    epsilon     = sigma*rnorm(length(X))
    Y = mu(X) + epsilon
    print(ggplot() + 
      geom_line(aes(x=x, y=mu(x)), alpha=.5) + 
        geom_point(aes(x=X, y=Y), alpha=.2))  
}



For each curve $\mu$, we'll do the following.

1.  Sample $n=200$ some observations $(X_i,Y_i)$ with

    -   $X_i$ uniformly distributed on $[0,1]$
    -   $Y_i = \mu(X_i) + \varepsilon_i$ where $\varepsilon_i \sim N(0,\sigma^2)$ for $\sigma=.5$.

    And sample independent copies $(\tilde X_i,\tilde Y_i)$ from the same distribution.

2.  Use our code to find estimates $\hat\mu$ of the curve using variation budgets $B \in \{1/10,\ 1/4,\ 1/2,\ 1,\ 2,\ 4,\ 10\}$.

3.  For each of these estimates, calculate

    -   Training Error, $\frac1n\sum_{i=1}^n \{ \hat\mu(X_i) - Y_i \}^2$
    -   Test Error, $\frac1n\sum_{i=1}^n \{ \hat\mu(\tilde X_i) - \tilde Y_i \}^2$
    -   Population MSE, $\int_0^1 \{ \hat\mu(x) - \mu(x)\}^2 dx + \sigma^2$.  

    Adding $\sigma^2$ to the last error makes it comparable to the others, which are sensitive to the noise level.

We'll do it all 10 times and average the results to get a decent approximation to the expectation of each of these error measures. Let's run the blocks below to tabulate there errors. It will take a few minutes.

In [ ]:
training.error = function(mu.hat, d) { 
  mean((mu.hat(d$X)-d$Y)^2)
}
test.error     = function(mu.hat, d) { 
  mean((mu.hat(d$Xtest)-d$Ytest)^2)
}
population.mse = function(mu.hat, d) { 
  x=seq(0,1,by=.001)
  mean((mu.hat(x) - d$mu(x))^2) + sigma^2
}   
errors = list(training=training.error, 
              test=test.error, 
              population=population.mse)

Bs = c(.1,.25,.5,1,2,4,10)
models = Bs |> 
  map(\(B) \(X,Y) bvreg(X,Y,B)) |> 
  set_names(sprintf('tv=%1.2f', Bs))



In [ ]:
on_sample = function(f, mu, sigma, n) {
    X     = runif(n)
    Xtest = runif(n)
    epsilon     = sigma*rnorm(length(X))
    epsilontest = sigma*rnorm(length(X))
    Y = mu(X) + epsilon
    Ytest = mu(Xtest) + epsilontest
    dataset = list(mu=mu,X=X,Y=Y,Xtest=Xtest,Ytest=Ytest)
    f(dataset)
}

summarize_fit = function(models) {
  function(d) {
    map(models, \(fit.model) { 
      model = fit.model(d$X,d$Y)
      mu.hat = prediction.function(model)
      errors |> map(\(e) data.frame(error=e(mu.hat, d))) |>
                list_rbind(names_to='error.measure') |>
                mutate(B=model$B)
    }) |> list_rbind(names_to='model')
  }
} 

In [ ]:
tab = 1:10 |> map(\(rep) { 
  mus |> map(\(mu) 
  summarize_fit(models) |> 
  on_sample(mu, sigma, n)
  ) |> list_rbind(names_to='mu')
}) |> list_rbind(names_to='rep') 


Here's what we get.


In [ ]:

ggplot(tab, 
       aes(x=B, y=error, linetype=error.measure, shape=error.measure)) + 
  geom_point(alpha=.2, position=position_jitter(w=.1)) + 
  stat_summary(geom='line',  fun=mean) +
  stat_summary(geom='point', fun=mean) +
  facet_grid(cols=vars(mu))




### Why Test Error and Population MSE agree so well

What we see in this plot is that Test Error and $\text{Population MSE} + \sigma^2$ are very close. That means we can effective choose our variation budget $B$ to minimize Population MSE. 
That's something we can't compute in practice because we'd need knowledge of $\mu$ and the distribution of $X$ to do it, but because test error tracks it so well, we get the same effect by choosing $B$ to minimize test error.
Test Error is, in fact, an *unbiased* estimate of $\text{Population MSE} + \sigma^2$. This is a pretty straightforward calculation. Letting $\tilde E$ denote expectation conditional on the training observations ...

$$
\begin{aligned}
\tilde{E} \{ \tilde Y_i - \hat\mu(\tilde X_i)\}^2 
&= \tilde{E} \{\tilde \varepsilon_i + \mu(\tilde X_i) - \hat \mu(\tilde X_i)\}^2 \quad \text{ for } \quad \tilde \varepsilon_i = \tilde Y_i - \mu(\tilde X_i) \\ 
&= \tilde{E} \tilde\varepsilon_i^2 + \tilde{E} \{\mu(\tilde X_i) - \hat\mu(\tilde X_i)\}^2 +
  2\tilde{E} \tilde \varepsilon_i \{\mu(\tilde X_i) - \hat\mu(\tilde X_i)\} \\
&= \sigma^2 + \text{PMSE} + 0
\end{aligned}
$$

Unbiasedness is, of course, not the whole story. Some unbiased estimates are usually nowhere near the quantity they're estimating. But this isn't just an unbiased estimate. It's an *average* of $n$ independent terms, each of which is an unbiased estimate of the same quantity. 
Roughly how far apart, as a function of sample size $n$, would you expect them to be? Does what you say require the noise $\varepsilon_i$ to be independent of $X_i$ or is it true in greater generality?.

*Hint.* What is the expectation of adjusted test error conditional on the training observations? And what is the conditional variance?

## Overfitting

Explain why training error tends to zero as $B$ increases. Why does test error not have this problem?

# Model selection

Since the variation budget $B$ matters, it's important to choose it well. Ideally, we'd do it automatically. Give it a shot.
Propose and implement an automated approach to choosing the variation budget $B$ in the block below.

**Hint**. Do we have an error measure that is both sensible and computable? To keep things simple, I'll ask that you choose from a finite set of options $\{B_1 \ldots B_k\}$. 

In [ ]:
select.budget = function(X,Y,Bs) {
  max(Bs) # You can do better than this
}


Let's see how well this works. We'll look at error varies as a function of sample size for bounded variation regression with a few fixed budgets, as well as with a budget chosen by this procedure.
To do this, we'll make a table of errors at different sample sizes, then do some visualization. Since everything is random, we'll do all this 10 times, average the results, and take a look at how variable these results are. 

Run the block below to make the table. It may take a few minutes. Keep in mind that if you haven't finished some of the previous exercises, the results you'll get might look pretty odd.
For example, if your version of the function `tvreg` above is really just monotone regression, varying the budget won't do anything. And if you haven't implemented the model selection procedure, you'll just get a comparison of the fixed budget-approaches. 

In [ ]:
on_samples = function(f, mu, sigma, ns) {
    X     = runif(max(ns))
    Xtest = runif(max(ns))
    epsilon     = sigma*rnorm(length(X))
    epsilontest = sigma*rnorm(length(X))
    Y = mu(X) + epsilon
    Ytest = mu(Xtest) + epsilontest
    ns |> map(\(n) {
      dataset = list(mu=mu,X=X[1:n],Y=Y[1:n],Xtest=Xtest[1:n],Ytest=Ytest[1:n])
      f(dataset) |> mutate(n=n)
    }) |> list_rbind()
}
bvmodel = function(B) { function(X,Y) { bvreg(X, Y, B=B) }}
bvselected = function(Bs) {
  function(X,Y) { 
    B = select.budget(X,Y,Bs)
    bvreg(X,Y,B=B)
  }
}
models  = list(bv0.5      = bvmodel(.5),
               bv1        = bvmodel(1),
               bv2        = bvmodel(2),
               bv4        = bvmodel(4),
               bvselected = bvselected(c(.5,1,2,4)))


ns = c(25,50,100)
grid = expand_grid(rep=1:10, mu=names(mus))
tab = grid |> 
  pmap(\(rep, mu) on_samples(summarize_fit(models), mus[[mu]], sigma, ns) |> mutate(rep=rep, mu=mu)) |>
  list_rbind() 


Now run this block to plot the error curves for each of our models. What we're seeing in bold color are the means of our error measures plus/minus one standard error. This gives us a sense of which method performs best of average. To illustrate how random performance is---how much it varies when we repeat the experiment---we'll also plot the error curves for each of our 10 trials faintly. This is called a *spaghetti plot*. They tend to be a bit of a mess, but they do give you an opportunity to get a gestalt impression of what's going on without committing to any particular summary statistic.

In [ ]:
for(mu in names(mus)) {
  plot.data = tab[tab$mu == mu, ]
  print(ggplot(plot.data, aes(x=n, y=error, color=model)) + 
    geom_line(aes(group=interaction(model,rep), color=model), alpha=.3, linewidth=.5) +
    stat_summary(geom='line',  fun=mean, position=position_dodge(width = 3)) +
    stat_summary(geom='pointrange', fun.data=mean_se, position=position_dodge(width=3)) +
    facet_grid(cols=vars(error.measure)) + ggtitle(mu))
}


How does your model selection code perform relative to the other methods? Does it tend to come in first, second, etc in terms of its accuracy? Why? 